## Tarea

Implementar un sistema automático reconocedor de caracteres trazados con el mouse, los caracteres son os dígitos del 0 al 9. Por cada dígito, entrenar un modelo oculto de Markov discreto del tipo izquierda-derecha utilizando varios ejemplos de entrenamiento por cada modelo (Ej 3). El clasificador deberá decidir por el carácter asociado al modelo que tenga la mayor probabilidad de haber producido la secuencia de observaciones (secuencia de pendientes cuantificadas) del carácter que se pretende reconocer. Utilizar los algoritmos que vimos en clase (Algoritmo Forward para clasificar y algoritmo Baum-Welch para entrenar), esto quiere decir, no utilizar una librería de modelos ocultos de Markov (A menos que sea HTK o que no usen modelos discretos sino semi-contínuos de Markov). El programa debe leer los movimientos del mouse y reportar el carácter reconocido en cuanto se termine el trazo de dicho carácter.



In [18]:
# TODO ajustar estos valores.


# Arreglos de observaciones para entrenar
# 0
dig0 = [10, 9, 9, 8, 7, 6, 5, 5, 3, 1, 1, 1, 15, 1, 14, 13, 13, 12]
dig00 = [9, 8, 8, 6, 5, 4, 2, 1, 15, 15, 14, 13, 13, 12]
dig000 = [9, 8, 7, 6, 5, 5, 4, 4, 3, 2, 1, 1, 1, 14, 12, 12, 12, 11, 11, 10]
# 1
dig1 = [5, 5, 5, 5, 5, 5, 5, 5]
dig01 = [5, 5, 5, 5, 5, 5, 4]
dig001 = [5, 5, 5, 5, 5, 5, 5, 5, 6]
# 2
dig2 = [15, 15, 16, 3, 6, 7, 7, 6, 6, 3, 1, 1, 1]
dig02 = [14, 15, 16, 1, 3, 5, 6, 7, 7, 7, 7, 7, 6, 2, 1, 1, 1, 1]
dig002 = [14, 15, 1, 3, 4, 5, 6, 7, 7, 7, 7, 2, 2, 2]
# 3
dig3 = [1, 1, 1, 5, 7, 8, 8, 1, 1, 3, 5, 7, 8, 9]
dig03 = [1, 1, 1, 1, 8, 8, 8, 8, 2, 1, 2, 3, 6, 7, 8, 9, 9]
dig003 = [1, 1, 2, 5, 7, 8, 9, 1, 1, 1, 2, 4, 6, 7, 8, 9, 9]
# 4
dig4 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15,
        15, 15, 15, 15, 6, 5, 5, 5, 5, 5, 5, 5]
dig04 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15, 15, 15, 15, 3, 5, 5, 5, 5, 5]
dig004 = [8, 9, 9, 9, 9, 16, 1, 16, 14, 14, 14, 14, 2, 5, 5, 5, 5, 5]
# 5
dig5 = [9, 9, 9, 9, 9, 8, 4, 5, 1, 1, 1, 2, 3, 3, 4, 5, 7, 8, 9, 10, 10]
dig05 = [9, 9, 9, 8, 5, 4, 1, 1, 1, 3, 3, 4, 5, 7, 8, 9, 9]
dig005 = [9, 9, 9, 8, 5, 5, 1, 1, 1, 2, 2, 4, 5, 6, 8, 9, 10, 11, 12]
# 6
dig6 = [8, 7, 7, 6, 5, 4, 2, 1, 16, 14, 13, 10, 8, 8, 7]
dig06 = [9, 8, 7, 6, 5, 4, 4, 3, 2, 2, 1, 15, 13, 12, 10, 9, 8, 7, 6]
dig006 = [8, 8, 7, 6, 5, 5, 4, 3, 2, 1, 15, 13, 10, 8]
# 7
dig7 = [1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7]
dig07 = [1, 1, 1, 1, 1, 1, 3, 6, 6, 6, 6, 5, 6, 6, 6, 6]
dig007 = [1, 1, 1, 1, 1, 3, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6]
# 8
dig8 = [9, 8, 5, 2, 1, 2, 3, 4, 6, 9, 10, 11, 12, 13, 15, 15, 15, 11]
dig08 = [11, 9, 8, 7, 5, 4, 1, 1, 1, 2, 3, 4, 6, 7,
         9, 9, 10, 10, 12, 13, 14, 15, 16, 16, 15, 14]
dig008 = [10, 9, 9, 6, 5, 4, 2, 2, 2, 2, 3, 4, 4, 5, 7, 9,
          9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 14, 13, 11, 10]
# 9
dig9 = [10, 8, 6, 5, 2, 1, 13, 13, 5, 5, 5, 5, 5, 5]
dig09 = [9, 8, 6, 5, 3, 1, 15, 13, 13, 5, 5, 5, 5, 5, 5, 5]
dig009 = [9, 9, 7, 4, 2, 16, 13, 14, 5, 5, 4, 4, 5, 5, 5]


In [19]:
# fuciones utiles
import numpy as np


def stochastic(matrix):
    """Recives a matrix and returns it row stochastic, row sum equals 1"""
    return matrix/matrix.sum(axis=1)[:, None]


def smoothMatx(mat, smoothing=1e-5):
    """ sets floor value to 1e-5, smoothing value can be provided"""
    
    mat[mat == 0] = smoothing
    return mat


In [20]:
def logForward(V, a, b, pi):
    """Forward returns logprob"""
    T = V.shape[0]
    scale = np.zeros(T)
    alpha = np.zeros((T, a.shape[0]))

    alpha[0, :] = pi * b[:, V[0]]
    scale[0] = alpha[0, :].sum()
    # alpha scaled
    alpha[0] = alpha[0]/scale[0]
    # Induction steps
    for t in range(1, T):
        alpha_hat = alpha[t - 1].dot(a) * b[:, V[t]]
        # alpha scaling 
        scale[t] = alpha_hat.sum()
        alpha[t] = alpha_hat / scale[t]
    # print(alpha)
    return np.log(scale).sum()

def probforward(V, a, b, pi):
    """Forward returns prob, it may ocour underflow"""
    p = 1
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, V.shape[0]):
        probability_of_observation = 0  
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            probability_of_observation += alpha[t, j]  
        p = p * probability_of_observation  
    # print(alpha)
    return p  


def forward(V, a, b, pi):
    T = V.shape[0]
    alpha = np.zeros((T, a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, T):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]

    return alpha


def backward(V, a, b):
    T = V.shape[0]
    beta = np.zeros((T, a.shape[0]))
    beta[T - 1] = np.ones((a.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(T - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])

    return beta


def baum_welch(V, a, b, pi, n_iter=100):
    T = V.shape[0]
    M = a.shape[0]

    for n in range(n_iter):
        alpha = forward(V, a, b, pi)

        # print(alpha)
        beta = backward(V, a, b)

        # xi array m*m*t
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(
                np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * \
                    b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        # print(gamma)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T element in gamma
        gamma = np.hstack(
            (gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))

    return {"a": a, "b": b}


In [21]:
# Recreamos el experimento de la presentación de Dr. Sung-Jung Cho de HMM

# Preflight Test

# obs 0 demo
V = np.array([7, 7, 6, 5, 5, 4, 4, 3, 1, 0, 15, 15, 14, 13, 12, 12, 11, 10, 9])

# Transition Probabilities
a = np.random.rand(3, 3)
a[1, 0] = 0
a[2, 0] = 0
a[2, 1] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(3, 16)
b = stochastic(b)
pi = np.array((1, 0, 0))

# train model with demo 0 samples
fit1 = baum_welch(V, a, b, pi, n_iter=100)
print(f'Trans Mat: \n {fit1["a"]}')
print(f'Obs Mat: \n {fit1["b"]}')


a = fit1["a"]
b = fit1["b"]

# B matrix smoothing, change 0's to 1e-5
bsmooth = smoothMatx(b) 

# Classification with demos of 0 and 1 

p0 = probforward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi)  
print(f"0 prob: {p0}")
p1 = probforward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)  
print(f"1 prob: {p1}")


logp0 = logForward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi) 
print(f"0 Logprob: {logp0}")

logp1 = logForward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)
print(f"1 Logprob: {logp1}")


Trans Mat: 
 [[0.66666668 0.33333332 0.        ]
 [0.         0.85711472 0.14288528]
 [0.         0.         1.        ]]
Obs Mat: 
 [[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 4.69240089e-008 3.33333318e-001 6.66666635e-001
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [1.42688357e-001 1.42885277e-001 0.00000000e+000 1.42885277e-001
  2.85770554e-001 2.85770534e-001 6.74743458e-025 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 2.86758656e-129]
 [1.53106406e-004 9.61896785e-105 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 1.11094099e-001 1.11094099e-001 1.11094099e-001
  2.22188199e-001 1.11094099e-001 1.11094099e-001 2.22188199e-001]]
0 prob: 6.4336295929061716e-195
1 prob: 1.7782665505888298e-28
0 Logpro

### Ventana que captura los trazos y emite las observaciones.

In [22]:
from tkinter import *
import math
import numpy as np
app = Tk()
app.geometry("400x400")


canvas = Canvas(app, bg='white')
canvas.pack(anchor='nw', fill='both', expand=1)

xcords = []
ycords = []


def get_x_and_y(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y


def draw(event):
    global lasx, lasy
    xcords.append(lasx)
    ycords.append(lasy)
    canvas.create_line((lasx, lasy, event.x, event.y),
                       fill='red',
                       width=2)
    lasx, lasy = event.x, event.y


def release(event):
    print("Click release!")
    genObser(xcords, ycords)


def genObser(xlist, ylist):
    """Generate observation sequence"""
    xpurge, ypurge = [], []
    deglist = []
    obs = []
    for i in range(0, len(xlist), 20):
        xpurge.append(xlist[i])
        ypurge.append(ylist[i])

    for i in range(len(xpurge)-1):
        dy = ypurge[i]-ypurge[i+1]
        dx = xpurge[i]-xpurge[i+1]
        myradians = math.atan2(dy, dx)
        mydegrees = math.degrees(myradians)
        deglist.append(mydegrees)
        # print(mydegrees)

    for i in deglist:
        # 0
        if i >= 168.75 and i <= 180:
            obs.append(0)
        if i <= -168.75:
            obs.append(0)
        # 1
        if i > -168.75 and i < -146.25:
            obs.append(1)
        # 2
        if i > -146.25 and i < -123.75:
            obs.append(2)
        # 3
        if i > -123.75 and i < -101.25:
            obs.append(3)
        # 4
        if i > -101.25 and i < -78.75:
            obs.append(4)
        # 5
        if i > -78.75 and i < -56.25:
            obs.append(5)
        # 6
        if i > -56.25 and i < -33.75:
            obs.append(6)
        # 7
        if i > -33.75 and i < -11.25:
            obs.append(7)
        # 8
        if i >= 0 and i < 11.25:
            obs.append(8)
        if i < 0 and i > -11.25:
            obs.append(8)
        # 9
        if i >= 11.25 and i < 33.75:
            obs.append(9)
        # 10
        if i >= 33.75 and i < 56.25:
            obs.append(10)
        # 11
        if i >= 56.25 and i < 78.75:
            obs.append(11)
        # 12
        if i >= 78.75 and i < 101.25:
            obs.append(12)
        # 13
        if i >= 101.25 and i < 123.75:
            obs.append(13)
        # 14
        if i >= 123.75 and i < 146.25:
            obs.append(14)
        # 15
        if i >= 146.25 and i < 168.75:
            obs.append(15)
    print(f'Obs Sequence: {obs}')


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()


solto boton
Secuencia de observaciones [15, 0, 0, 1, 3, 6, 7, 8, 15, 0, 1, 2, 4, 5, 6, 7, 8, 9]
